## Filtering of the raw data

This notebook loads and cleans the raw log and result files generated from phylogenetic inference runs. It extracts key information such as iteration costs, topology moves, runtime, sequence number, alignment length, and tree structures. The notebook also computes RF distances between start and final trees and prepares the cleaned data for further statistical analysis.

**Library import**

In [ ]:
import re
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
from Bio import AlignIO
from io import StringIO
from Bio import Phylo
from io import StringIO
from Bio.Phylo.TreeConstruction import _Matrix
from itertools import combinations

**Import of data**

In [2]:
# Tree import
start_tree_path = r"1762246214192778_out/1762246214192778_start_tree.newick"
final_tree_path = r"1762246214192778_out/1762246214192778_tree.newick"

# Read file content
file_path = '1762246214192778_out/1762246214192778.log'
log_content = ""

try:
    # Specify encoding (encoding='utf-8') to avoid decoding errors
    with open(file_path, 'r', encoding='utf-8') as f:
        log_content = f.read()
    print(f"File '{file_path}' successfully read using UTF-8 encoding.")
except FileNotFoundError:
    print(f"ERROR: The file '{file_path}' was not found.")
    print("Please ensure that the file is in the same directory as the script or adjust the path accordingly.")
    exit()
except Exception as e:
    # If UTF-8 fails, 'latin-1' is often the last resort
    if 'UnicodeDecodeError' in str(e):
        try:
            with open(file_path, 'r', encoding='latin-1') as f:
                log_content = f.read()
            print(f"NOTE: UTF-8 failed, but the file '{file_path}' was successfully read using LATIN-1.")
        except Exception as e_latin:
            print(f"CRITICAL ERROR: Neither UTF-8 nor LATIN-1 could read the file: {e_latin}")
            exit()
    else:
        print(f"ERROR while reading the file: {e}")
        exit()

ERROR: The file '1762246214192778_out/1762246214192778.log' was not found.
Please ensure that the file is in the same directory as the script or adjust the path accordingly.


**Initialize data structures**

In [3]:
model_iterations = []
model_costs = []
topo_steps = []
topo_costs = []
spr_counter = 0

**Define regular expressions**

In [4]:
# Main ML optimization (e.g., branch length optimization)
model_pattern = re.compile(r"phylo::optimisers::blen_optimiser\s+Iteration: (\d+), current cost: (-?\d+\.\d+)")

# Topology optimization (initial and accepted SPR moves)
topo_initial_pattern = re.compile(r"phylo::optimisers::topo_optimiser Initial cost: (-?\d+\.\d+)")
topo_spr_pattern = re.compile(r"SPR move applied, new cost (-?\d+\.\d+)")

# Timestamp pattern (HH:MM:SS)
time_pattern = re.compile(r"^(\d{2}:\d{2}:\d{2})")

**New patterns for sequence count and alignment length**

In [5]:
seq_pattern = re.compile(
    r"(?:Loaded alignment with|Number of sequences:|Number of sequence\(s\):?)\s*(\d+)\s*(?:sequence\(s\)?|sequences?)?",
    re.IGNORECASE
)
align_pattern = re.compile(
    r"(?:and\s*)?(\d+)\s*(?:sites|positions|columns|bp|alignment length)",
    re.IGNORECASE
)

# --- New variables ---
sequence_number = None
alignment_length = None

**Extract data**

In [6]:
start_time = None
end_time = None

for line in log_content.splitlines():

    # Extract model optimization 
    match_model = model_pattern.search(line)
    if match_model:
        iteration = int(match_model.group(1))
        cost = float(match_model.group(2))
        model_iterations.append(iteration)
        model_costs.append(cost)

    # Extract sequence number
    if sequence_number is None:
        match_seq = seq_pattern.search(line)
        if match_seq:
            sequence_number = int(match_seq.group(1))

    # Extract alignment length 
    if alignment_length is None:
        match_align = align_pattern.search(line)
        if match_align:
            alignment_length = int(match_align.group(1))

    # Extract topology optimization 
    # Store the initial cost only once
    match_initial = topo_initial_pattern.search(line)
    if match_initial and not topo_costs:
        cost = float(match_initial.group(1))
        topo_steps.append(spr_counter)
        topo_costs.append(cost)

    # Extract accepted SPR moves
    match_spr = topo_spr_pattern.search(line)
    if match_spr:
        spr_counter += 1
        cost = float(match_spr.group(1))
        topo_steps.append(spr_counter)
        topo_costs.append(cost)

    # Detect start and end time
    if "JATI run started" in line:
        match = time_pattern.match(line)
        if match:
            start_time = datetime.strptime(match.group(1), "%H:%M:%S")
    elif "Final log-likelihood" in line:
        match = time_pattern.match(line)
        if match:
            end_time = datetime.strptime(match.group(1), "%H:%M:%S")

**Compute total runtime**

In [ ]:
total_runtime = None
if start_time and end_time:
    total_runtime = (end_time - start_time).total_seconds()
    print(f"Total runtime: {total_runtime:.1f} seconds ({total_runtime/60:.1f} minutes)")
else:
    print("WARNING: No start/end time found in the log — runtime cannot be calculated.")
    total_runtime = 1.0  # FalSlback to avoid division by zero

**Summarize results**

In [8]:
if model_costs and topo_costs:
    print(f"\n--- Summary ---")
    print(f"Final log-likelihood (ML) value after parameter optimization: {-model_costs[-1]:.4f}")
    # print(f"Final log-likelihood (ML) value after topology optimization: {-topo_costs[-1]:.4f}")

**Number of sequences and Alignment length**

In [1]:
alignment_path = "C:/A.Fächer/5. Semester/A. Project 2/benchmark-datasets/benchmark-datasets/aa/medium/boroa6_OG303_gene354.fasta"

# Try UTF-8; if that fails, fall back to Latin-1
try:
    with open(alignment_path, 'r', encoding='utf-8') as f:
        content = f.read()
except UnicodeDecodeError:
    with open(alignment_path, 'r', encoding='latin-1') as f:
        content = f.read()

# Pass content to AlignIO
alignment = AlignIO.read(StringIO(content), "fasta")

sequence_number = len(alignment)
alignment_length = alignment.get_alignment_length()

print(f"Number of sequences: {sequence_number}")
print(f"Alignment length: {alignment_length}")

NameError: name 'AlignIO' is not defined

**Calculate RF Distance**

In [ ]:
def robinson_foulds(tree1, tree2):
    """
    Computes the (non-normalized and normalized) RF distance between two trees.
    """
    # Extract splits (clades)
    def get_splits(tree):
        terminals = tree.get_terminals()
        n = len(terminals)
        splits = set()
        for clade in tree.find_clades(order='postorder'):
            if clade.is_terminal():
                continue
            split = frozenset([t.name for t in clade.get_terminals()])
            if 0 < len(split) < n:
                splits.add(split)
        return splits

    splits1 = get_splits(tree1)
    splits2 = get_splits(tree2)

    rf_distance = len(splits1.symmetric_difference(splits2))
    max_splits = len(splits1) + len(splits2)
    normalized_rf = rf_distance / max_splits if max_splits > 0 else 0

    return rf_distance, normalized_rf

# Read Newick trees 
try:
    tree1 = Phylo.read(start_tree_path, "newick")
    tree2 = Phylo.read(final_tree_path, "newick")
except Exception as e:
    print(f"Error reading trees: {e}")
    exit()

# Compute RF distance 
rf_distance, normalized_rf = robinson_foulds(tree1, tree2)

print("\n--- RF Distance ---")
print(f"Robinson–Foulds Distance: {rf_distance}")
print(f"Normalized RF Distance: {normalized_rf:.4f}")

# Tree info
print(f"\nNumber of taxa in Tree 1: {len(tree1.get_terminals())}")
print(f"Number of taxa in Tree 2: {len(tree2.get_terminals())}")

Error reading trees: name 'Phylo' is not defined


NameError: name 'tree1' is not defined

: 